In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mage
import keras 
from sklearn.model_selection import train_test_split
from keras.layers import *
from keras.models import *
from keras import backend as K
from keras.regularizers import l2
from sklearn.preprocessing import MinMaxScaler
from skimage.color import rgb2gray
from skimage.util import img_as_ubyte
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
import itertools

In [ ]:
#Data Path
base_path = '../input/age-gender-and-ethnicity-face-data-csv'
output_path = './'

In [ ]:
get_csv = pd.read_csv(base_path+'/age_gender.csv')
data_csv = pd.DataFrame(get_csv)
data_csv['pixels']=data_csv['pixels'].apply(lambda x:  np.array(x.split(), dtype="float32"))
img_data = np.array(data_csv['pixels'].tolist())
img_data = img_data.reshape(img_data.shape[0], 48,48,1)
size = len(img_data)

In [ ]:
images = []
ages = []
genders = []
for idx, img in enumerate(img_data[:]):
    if(idx%5000==0):print(idx)
    image = img
    image = image.reshape((image.shape[0],image.shape[1]))
    images.append(image)
    ages.append(data_csv['age'][idx])
    genders.append(data_csv['gender'][idx])

In [ ]:
np_genders = np.array(genders)
x_genders = 'Pria', 'Wanita' #list(set(genders))
y_genders = [np_genders[np_genders==0].shape[0], np_genders[np_genders==1].shape[0]]
plt.bar(x_genders,y_genders)
plt.show()
print("Max value:",max(y_genders))

In [ ]:
x_ages = list(set(ages))
y_ages = [ages.count(i) for i in x_ages]
plt.bar(x_ages,y_ages)
plt.show()
print("Max value:",max(y_ages))

In [ ]:
def display(img):
    plt.imshow(img)
    plt.set_cmap('gray')
    plt.show()

def age_group(age):
    if age <= 1:
        return 0
    elif age <= 14:
        return 1
    elif age <= 44:
        return 2
    elif age <= 64:
        return 3
    else:
        return 4
    # kategori umur dari https://unstats.un.org/unsd/publication/SeriesM/SeriesM_74e.pdf#:~:text=Thus%2C%20%20single%20%20years%20%20are%20,%20groups%20%20for%20%20ages%20%205-74.

def get_age(distr):
    distr = np.argmax(distr, axis=1)
    if distr == 0:
      return "<1"
    if distr == 1:
      return "1-14"
    if distr == 2:
      return "15-44"
    if distr == 3:
      return "45-64"
    if distr == 4:
      return "65>"
    else: 
      return "Unknown"
    
def get_gender(prob):
    prob = np.argmax(prob, axis=1)
    if prob == 0:
      return "Male"
    elif prob == 1:
      return "Female"

def get_result(sample):
    gray = rgb2gray(sample)
    norm = (gray-np.min(gray))/(np.max(gray)-np.min(gray))
    img8bit = img_as_ubyte(norm)
    val = model.predict(np.array([img8bit]))
    age = get_age(val[0])
    gender = get_gender(val[1])
    print("Values:",val,"\nPredicted Gender:",gender,"Predicted Age:",age)

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
# pre processingpre processing
target = [[],[]]
features = np.zeros((size,48,48),dtype = 'float32')
for i in range(size):
    if(i%5000==0):print(i)
    target[0].append(age_group(int(ages[i]))) #age_group(int(ages[i])) / 5
    target[1].append(int(genders[i])) # target[i,1] = int(genders[i])
    gray = rgb2gray(images[i])
    norm = (gray-np.min(gray))/(np.max(gray)-np.min(gray))
    img8bit = img_as_ubyte(norm)
    features[i] = img8bit
print('Before grayscale')
imgplot = plt.imshow(images[3996])
plt.show()
# features = rgb2gray(features)
print('After grayscale')
display(features[3996])
print("|  Age:",ages[3996])

features = np.array([features])
features = features.reshape(size, 48, 48, 1)
features.shape

In [ ]:
labels = np.array(target)
labels.reshape(size,2)

labels1 = tf.keras.utils.to_categorical(labels[0])
labels2 = tf.keras.utils.to_categorical(labels[1])
labels1 = np.array(labels1)
labels2 = np.array(labels2)

x_train, x_test, y_train, y_test = train_test_split(features, labels1, test_size=0.2, shuffle = True, random_state=1)
x_train2, x_test2, y_train2, y_test2 = train_test_split(features, labels2, test_size=0.2, shuffle = True, random_state=1)
print("Samples in Training:",x_train.shape[0])
print("Samples in Testing:",x_test.shape[0])

In [ ]:
inputs = Input(shape=(48,48,1))
conv1 = Conv2D(6, kernel_size=(3, 3),activation='relu',padding = 'same',strides=(1, 1),kernel_regularizer=l2(0.001))(inputs)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = Conv2D(12, kernel_size=(3, 3),activation='relu',padding = 'same',strides=(1, 1),kernel_regularizer=l2(0.001))(pool1)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
conv3 = Conv2D(24, kernel_size=(3, 3),activation='relu',padding = 'same',strides=(1, 1),kernel_regularizer=l2(0.001))(pool2)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
conv4 = Conv2D(48, kernel_size=(3, 3),activation='relu',padding = 'same',strides=(1, 1),kernel_regularizer=l2(0.001))(pool3)
pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
conv5 = Conv2D(96, kernel_size=(3, 3),activation='relu',padding = 'same',strides=(1, 1),kernel_regularizer=l2(0.001))(pool4)
pool5 = MaxPooling2D(pool_size=(2, 2))(conv5)
x = Dropout(0.25)(pool5)
flat = Flatten()(x)

dropout = Dropout(0.25)
age_model = Dense(96, activation='relu')(flat)
age_model = dropout(age_model)
age_model = Dense(48, activation='relu')(age_model)
age_model = dropout(age_model)
age_model = Dense(24, activation='relu')(age_model)
age_model = dropout(age_model)
age_model = Dense(12, activation='relu')(age_model)
age_model = dropout(age_model)
age_model = Dense(6, activation='relu')(age_model)
age_model = dropout(age_model)
age_model = Dense(5, activation='sigmoid')(age_model)

gender_model = Dense(96, activation='relu')(flat)
gender_model = dropout(gender_model)
gender_model = Dense(48, activation='relu')(gender_model)
gender_model = dropout(gender_model)
gender_model = Dense(24, activation='relu')(gender_model)
gender_model = dropout(gender_model)
gender_model = Dense(12, activation='relu')(gender_model)
gender_model = dropout(gender_model)
gender_model = Dense(6, activation='relu')(gender_model)
gender_model = dropout(gender_model)
gender_model = Dense(2, activation='sigmoid')(gender_model)

model = Model(inputs=inputs, outputs=[age_model, gender_model])
model.compile(optimizer = 'adam', loss =['mse','binary_crossentropy'],metrics=['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])#tf.keras.metrics.Precision(), tf.keras.metrics.Recall() #loss =['mae','mse','binary_crossentropy']

from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
fle_s='Age_sex_detection.h5' #model(input_shape=(64,64,1))
checkpointer = ModelCheckpoint(fle_s, monitor='val_loss',verbose=1,save_best_only=True,save_weights_only=False, mode='auto',save_freq='epoch')
Early_stop=tf.keras.callbacks.EarlyStopping(patience=75, monitor='val_loss',restore_best_weights=True),
callback_list=[checkpointer,Early_stop]

h = model.fit(x_train,[y_train,y_train2],validation_data=(x_test,[y_test,y_test2]),epochs = 100, batch_size=96,shuffle = True,callbacks=[callback_list])

In [ ]:
%matplotlib inline

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

tf.keras.utils.plot_model(model, to_file='model.png')
img = mpimg.imread('model.png')

plt.figure(figsize=(40, 30))
plt.imshow(img)

In [ ]:
history = h
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model accuracy')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
loss = np.mean(history.history['loss'])
print("%s: %.2f%%" % ('loss',(loss)))

In [ ]:
plt.plot(history.history['dense_5_accuracy'])
plt.plot(history.history['val_dense_5_accuracy'])
plt.title('model age accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

acc_train = np.mean(history.history['dense_5_accuracy'])
acc_test = np.mean(history.history['val_dense_5_accuracy'])
print("%s: %.2f%%" % ('train_age_mean_accuracy',(acc_train*100)))
print("%s: %.2f%%" % ('test_age_mean_accuracy',(acc_test*100)))

In [ ]:
plt.plot(history.history['dense_11_accuracy'])
plt.plot(history.history['val_dense_11_accuracy'])
plt.title('model gender accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

acc_train = np.mean(history.history['dense_11_accuracy'])
acc_test = np.mean(history.history['val_dense_11_accuracy'])
print("%s: %.2f%%" % ('train_gender_mean_accuracy',(acc_train*100)))
print("%s: %.2f%%" % ('test_gender_mean_accuracy',(acc_test*100)))

In [ ]:
# Evaluate the model on test set
score = model.evaluate(x_test,[y_test,y_test2])

y_pred = model.predict(x_test)
# Confusion Matrix
from sklearn.metrics import confusion_matrix
cx=confusion_matrix( np.argmax(y_test, axis=1), np.argmax(y_pred[0], axis=1))
cx2=confusion_matrix( np.argmax(y_test2, axis=1), np.argmax(y_pred[1], axis=1))

plot_confusion_matrix(cx,classes=['<1', '1-14', '15-44', '45-64', '65>'])
plt.show()
plot_confusion_matrix(cx2,classes=['male', 'female'])
plt.show()

# Accuracy
from sklearn.metrics import accuracy_score
acc=accuracy_score(np.argmax(y_test,axis=1), np.argmax(y_pred[0],axis=1))
acc2=accuracy_score(np.argmax(y_test2, axis=1), np.argmax(y_pred[1], axis=1))
# Recall
from sklearn.metrics import recall_score
re=recall_score(np.argmax(y_test,axis=1), np.argmax(y_pred[0],axis=1), average=None)
re2=recall_score(np.argmax(y_test2,axis=1), np.argmax(y_pred[1],axis=1), average=None)
# # Precision
from sklearn.metrics import precision_score
pre=precision_score(np.argmax(y_test,axis=1), np.argmax(y_pred[0],axis=1), average=None)
pre2=precision_score(np.argmax(y_test2,axis=1), np.argmax(y_pred[1],axis=1), average=None)

print("accuracy age: ", acc)
print("recall age: ", re)
print("precision age: ",  pre)
print("accuracy gender: ", acc2)
print("recall gender: ", re2)
print("precision gender: ",  pre2)

In [ ]:
indexes = [3000,69,500,9900,1900,1278,2,22210,4032]
for idx in indexes:
    sample = images[idx]
    img = images[idx]
    imgplot = plt.imshow(img)
    plt.show()
    #display(sample)
    print("Actual Gender:",genders[idx],"Age:",ages[idx])
    res = get_result(sample)